# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

## Text Preprocessing
### reference
> * [Google guide](https://developers.google.com/machine-learning/guides/text-classification/step-3)
> * N-grams
>> * [scikit-learn working with text data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#)
>> * [scikit-learn text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
>> * [한글 자료](https://datascienceschool.net/03%20machine%20learning/03.01.03%20Scikit-Learn%EC%9D%98%20%EB%AC%B8%EC%84%9C%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EB%8A%A5.html)
> * Sequence
>> * [keras text classification](https://keras.io/examples/nlp/text_classification_from_scratch/)
>> * [tensorflow text classification](https://www.tensorflow.org/tutorials/keras/text_classification)

---
#### 

### 0. 라이브러리 설치 및 불러오기

In [1]:
## import sklearn
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
mecab = Mecab()
#fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

[]
[]


### 1.  데이터 가져오기

In [2]:
# 데이터를 가져옵니다.
train_filename = '/aihub/workspace/spam.csv'
data = pd.read_csv(train_filename)
data.head()

,text,label
0,[Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA,spam
1,[Web발신]골든브릿지에서손실을보셨나요??http://bitly.kr/bRGtq[FW],spam
2,[Web발신][대//박]가(.원)입코(WN)드MEP.com,spam
3,[Web발신](광고)이정미님아직도주식하시나요http://pf.kakao.com/_u...,spam
4,[Web발신]송승용님사람이모이는곳에는이유가있습니다하루~정보공개is.gd/JsJP,spam


#### 1-1. processing label

In [3]:
# label 데이터를 수치형으로 변환합니다.
data['label'] = data['label'].map({'spam' : 1, "ham" : 0})

In [6]:
data['label'].unique()

array([1, 0])

In [9]:
data = data.dropna()

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20089 entries, 0 to 20099
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20089 non-null  object
 1   label   20089 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 470.8+ KB


In [11]:
data = data.drop_duplicates(['text'], keep='first', inplace=False, ignore_index=False)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18386 entries, 0 to 20099
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18386 non-null  object
 1   label   18386 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 430.9+ KB


### 2. Train Validation(Test) Split

In [13]:
from sklearn.model_selection import train_test_split

In [27]:
# train validation set으로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(data['text'], data['label'], test_size=0.2)

In [28]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((14708,), (3678,), (14708,), (3678,))

### 3. Vectorize texts

#### 3-1. N-grams Vectorize [참고](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#extracting-features-from-text-files)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
count_vect = CountVectorizer()

In [54]:
cv_x_train = count_vect.fit_transform(x_train)
cv_x_val = count_vect.transform(x_val)

In [55]:
cv_x_train.shape, cv_x_val.shape

((14708, 29499), (3678, 29499))

In [56]:
print(cv_x_train)

  (0, 4966)	1
  (0, 9499)	1
  (0, 25913)	1
  (0, 2696)	1
  (0, 4249)	1
  (1, 4966)	1
  (1, 23362)	1
  (1, 1771)	1
  (1, 3997)	1
  (1, 2080)	1
  (1, 675)	1
  (1, 49)	1
  (2, 4966)	1
  (2, 3127)	1
  (2, 6992)	1
  (2, 5336)	1
  (2, 14829)	1
  (2, 18527)	1
  (2, 20972)	1
  (2, 20971)	1
  (2, 8723)	1
  (3, 2696)	1
  (3, 11704)	1
  (3, 1662)	1
  (3, 408)	1
  :	:
  (14704, 28838)	1
  (14704, 6027)	1
  (14704, 7191)	1
  (14704, 10001)	1
  (14705, 3127)	1
  (14705, 6992)	1
  (14705, 17639)	1
  (14705, 6595)	1
  (14705, 15777)	1
  (14705, 2915)	1
  (14705, 15019)	1
  (14705, 7606)	1
  (14705, 23391)	1
  (14706, 4966)	1
  (14706, 1769)	1
  (14706, 3665)	1
  (14706, 4663)	1
  (14706, 25333)	1
  (14706, 1123)	1
  (14706, 20867)	1
  (14707, 6992)	1
  (14707, 3039)	1
  (14707, 14233)	1
  (14707, 9282)	1
  (14707, 3723)	1


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    """Vectorizes texts as n-gram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

    # Arguments
        train_texts: list, training text strings.
        train_labels: np.ndarray, training labels.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val: vectorized training and validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

In [62]:
ngram_x_train, ngram_x_val = ngram_vectorize(x_train, y_train, x_val)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [63]:
ngram_x_train.shape

(14708, 10410)

#### 3-2. Sequence Vectorize [참고](https://developers.google.com/machine-learning/guides/text-classification/step-3)

In [33]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

# Vectorization parameters
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Limit on the length of text sequences. Sequences longer than this
# will be truncated.
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):
    """Vectorizes texts as sequence vectors.

    1 text = 1 sequence vector with fixed length.

    # Arguments
        train_texts: list, training text strings.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val, word_index: vectorized training and validation
            texts and word index dictionary.
    """
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index

In [98]:
sequence_x_train, sequence_x_val, word_index = sequence_vectorize(x_train, x_val)

In [99]:
sequence_x_train[:1]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    1, 4863,  103,    3,
          39]], dtype=int32)

In [79]:
TfidVector = TfidfVectorizer()

In [93]:
sequence_x_train = TfidVector.fit_transform(x_train)
sequence_x_val = TfidVector.transform(x_val)

In [94]:
print(sequence_x_train)

  (0, 4249)	0.4178355550025851
  (0, 2696)	0.20983259916199754
  (0, 25913)	0.4869915088263504
  (0, 9499)	0.7299139404397907
  (0, 4966)	0.10699831394603691
  (1, 49)	0.42900020895547913
  (1, 675)	0.23828952315581392
  (1, 2080)	0.2663334367902144
  (1, 3997)	0.28063667543569043
  (1, 1771)	0.28698786998499026
  (1, 23362)	0.7183561099392967
  (1, 4966)	0.10530404794026392
  (2, 8723)	0.4233138463869956
  (2, 20971)	0.40598194474203847
  (2, 20972)	0.40598194474203847
  (2, 18527)	0.40598194474203847
  (2, 14829)	0.36405568326170157
  (2, 5336)	0.40598194474203847
  (2, 6992)	0.08268742157221205
  (2, 3127)	0.13526128984503297
  (2, 4966)	0.06205370979232093
  (3, 408)	0.5023297780243415
  (3, 1662)	0.698012919606757
  (3, 11704)	0.465474229394362
  (3, 2696)	0.20922834402105148
  :	:
  (14704, 28673)	0.4401511164176098
  (14704, 6992)	0.0859763063016612
  (14704, 3127)	0.140641295439577
  (14704, 4966)	0.06452189049817834
  (14705, 23391)	0.42393099603396645
  (14705, 7606)	0.423930

In [97]:
print(sequence_x_train.toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


### 4. Save data

#### 4-1. Save N-grams

In [46]:
# N-grams 방식으로 vectorize한 데이터의 shape을 확인해봅니다.
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
x_train_counts.shape

(14708, 29499)

#### 4-2. Save sequence

In [15]:
# Sequence 형태로 vectorize한 데이터의 shape을 확인해봅니다.
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.

X_train seq shape: (15066, 23)
X_val seq shape: (5023, 23)


In [47]:
sequence_x_train.shape

(14708, 23)

import pickle#### 4-3. Save label

In [ ]:
import pickle

In [4]:
# label 데이터의 shape을 확인하고 저장합니다.

In [102]:
with open('save_data.pickle', 'wb') as f:
    pickle.dump([data['text'], data['label']], f)
    pickle.dump([x_train, x_val], f)
    pickle.dump([y_train, y_val], f)
    pickle.dump([cv_x_train, cv_x_val], f)
    pickle.dump([ngram_x_train, ngram_x_val], f)
    pickle.dump([sequence_x_train, sequence_x_val, word_index], f)

In [65]:
with open('save_data.pickle', 'rb') as f:
    test_y_train, test_y_val = pickle.load(f)
    a = pickle.load(f)
    b = pickle.load(f)
    c = pickle.load(f)

In [68]:
test_y_train.shape, test_y_val.shape

((14708,), (3678,))

In [101]:
data['text'], data['label']

((18386,), (18386,))